# Clustering Crypto

In [76]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [87]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [88]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
r = requests.get(url)
data = r.json()

In [89]:
list(data.keys())

['Response', 'Message', 'Data', 'RateLimit', 'HasWarning', 'Type']

In [90]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
data = data['Data']
crypto_df = pd.DataFrame.from_dict(data)

In [82]:
coin_supply_df = pd.read_csv(file_path)
coin_supply_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [84]:
coin_supply_df.drop(columns=[
    'Unnamed: 0',
    'Algorithm',
    'IsTrading',
    'ProofType',
    'TotalCoinsMined'
], inplace=True)
coin_supply_df.head()

,CoinName,TotalCoinSupply
0,42 Coin,42
1,365Coin,2300000000
2,404Coin,532000000
3,SixEleven,611000
4,808,0


TotalCoinSupply was missing from the website.

In [92]:
crypto_df = crypto_df.T
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty,AlgorithmType
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Preprocessing

In [93]:
crypto_df.columns

Index(['Id', 'Url', 'ImageUrl', 'ContentCreatedOn', 'Name', 'Symbol',
       'CoinName', 'FullName', 'Description', 'AssetTokenStatus', 'Algorithm',
       'ProofType', 'SortOrder', 'Sponsored', 'Taxonomy', 'Rating',
       'IsTrading', 'TotalCoinsMined', 'CirculatingSupply', 'BlockNumber',
       'NetHashesPerSecond', 'BlockReward', 'BlockTime', 'AssetLaunchDate',
       'AssetWhitepaperUrl', 'AssetWebsiteUrl', 'MaxSupply', 'MktCapPenalty',
       'IsUsedInDefi', 'IsUsedInNft', 'PlatformType', 'BuiltOn',
       'SmartContractAddress', 'DecimalPoints', 'Difficulty', 'AlgorithmType'],
      dtype='object')

In [94]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df.drop(columns = [
    'AlgorithmType',
    'AssetLaunchDate',
    'AssetTokenStatus',
    'AssetWebsiteUrl',
    'AssetWhitepaperUrl',
    'BlockNumber',
    'BlockReward',
    'BlockTime',
    'BuiltOn',
    'CirculatingSupply',
    'ContentCreatedOn',
    'DecimalPoints',
    'Description',
    'Difficulty',
    'FullName',
    'Id',
    'ImageUrl',
    'IsUsedInDefi',
    'IsUsedInNft',
    'MaxSupply',
    'MktCapPenalty',
    'Name',
    'NetHashesPerSecond',
    'PlatformType',
    'Rating',
    'SmartContractAddress',
    'SortOrder',
    'Sponsored',
    'Symbol',
    'Taxonomy',
    'Url'
], inplace = True)


In [95]:
crypto_df.columns

Index(['CoinName', 'Algorithm', 'ProofType', 'IsTrading', 'TotalCoinsMined'], dtype='object')

Need to join TotalCoinSupply to crypto_df

In [40]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]

In [41]:
crypto_df.sample(10)

,CoinName,Algorithm,ProofType,IsTrading,TotalCoinsMined,MaxSupply
LSP,Lumenswap,N/A,N/A,True,999902965.92712,1000000000
STPT,STP Network,N/A,N/A,True,1942420283.027067,-1
YGG,Yield Guild Games,N/A,N/A,True,1000000000,-1
GIFT,GiftNet,X13,PoW,True,NaN,NaN
PRO,Propy,N/A,N/A,True,100000000,-1
ZCR,ZCore,Quark,PoS,True,0,0
LQTY,Liquity,N/A,N/A,True,100000000,-1
SAL,SalPay,N/A,N/A,True,NaN,NaN
JPYC,JPYC,N/A,N/A,True,1000000000,-1
GEM,Gems,N/A,N/A,True,8000000000,-1


In [42]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[crypto_df['Algorithm'] != 'N/A']

In [43]:
# Remove the "IsTrading" column
crypto_df.drop(columns = ['IsTrading'], inplace = True)

In [44]:
# Remove rows with at least 1 null value
crypto_df.isnull().sum()

CoinName              0
Algorithm             0
ProofType             0
TotalCoinsMined    1198
MaxSupply          1198
dtype: int64

In [45]:
crypto_df = crypto_df.dropna()
crypto_df.isnull().sum()

CoinName           0
Algorithm          0
ProofType          0
TotalCoinsMined    0
MaxSupply          0
dtype: int64

In [46]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]

In [47]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[crypto_df!='N/A']

In [48]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coinname_df = pd.DataFrame(
    data = crypto_df, columns = ['CoinName']
)
coinname_df.head()

,CoinName
NSR,NuShares
TRI,Triangles Coin
CMTC,CometCoin
CHAT,OpenChat
QRL,Quantum Resistant Ledger


In [49]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns = 'CoinName')


In [50]:
crypto_df.sample(30)

,Algorithm,ProofType,TotalCoinsMined,MaxSupply
ZEN,Equihash,PoW,11748606.25,21000000
CHAT,Scrypt,PoW/PoS,1000000000,-1
PACOCA,BEP-20 Token,NaN,77292913.005678,-1
WAXP,DPoS,NaN,3723328685.446701,-1
AR,SHA-256,PoW,64598643,66000000
DBC,NEP-5,NaN,10000000000,-1
QBIT,BEP-20 Token,NaN,1000000000000,-1
GAFI,BEP-20 Token,NaN,14965600,-1
VIA,Scrypt,PoW,23174976.588385,-1
XEC,SHA-256,PoW,18897833396789.378906,21000000000000


In [51]:
# Create dummy variables for text features
crypto_dummies= pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
crypto_dummies.head()

,TotalCoinsMined,MaxSupply,Algorithm_Autolykos,Algorithm_BEP-2,Algorithm_BEP-2 Token,Algorithm_BEP-20 Token,Algorithm_BEP2 Token,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake2B + SHA3,...,ProofType_PoW/PoSe,ProofType_PoW/nPoS,ProofType_ProgPoW/PoS,ProofType_Proof of Authority,ProofType_Proof-of-Work,ProofType_SPoS,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW,ProofType_dPoW/PoW
NSR,6168467233.8311,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TRI,190083.632215,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CMTC,872830,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHAT,1000000000,-1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
QRL,75321347.306076,105000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(crypto_dummies)
print(crypto_scaled[0:1])

[[-0.05832118 -0.06292184 -0.05832118 -0.10136061 -0.05832118 -0.92484913
  -0.05832118 -0.08261924 -0.05832118 -0.05832118 -0.10136061 -0.08261924
  -0.10136061 -0.05832118 -0.05832118 -0.19680547 -0.08261924 -0.05832118
  -0.05832118 -0.05832118 -0.19680547 -0.05832118 -0.05832118 -0.17739372
  -0.05832118 -0.05832118 -0.05832118 -0.05832118 -0.08261924 -0.05832118
  -0.05832118 -0.05832118 -0.05832118 -0.05832118 -0.05832118 -0.13130643
  -0.08261924 -0.05832118 -0.05832118 -0.05832118 -0.08261924  6.94022094
  -0.05832118 -0.05832118 -0.10136061 -0.08261924 -0.21470745 -0.08261924
  -0.05832118 -0.05832118 -0.05832118 -0.28387688 -0.05832118 -0.13130643
  -0.05832118 -0.08261924 -0.11724208 -0.05832118 -0.14408763 -0.08261924
  -0.05832118 -0.05832118 -0.05832118 -0.05832118 -0.05832118 -0.05832118
  -0.16695677 -0.05832118 -0.05832118 -0.08261924 -0.05832118 -0.05832118
   5.0811595  -0.05832118 -0.05832118 -0.05832118 -0.51584341 -0.30429031
  -0.05832118 -0.05832118 -0.05832118 

In [53]:
crypto_df.tail()

,Algorithm,ProofType,TotalCoinsMined,MaxSupply
ZEC,Equihash,PoW,11803847.2008,21000000
ETP,Ethash,NaN,81301153.33001,100000000
OXYC,DPoS,DPoS,1122382283.37,-1
PIVX,Quark,PoW/PoS,67563118.059209,-1
POA,Proof-of-Authority,PoA,293808782.623202,-1


### Reducing Dimensions Using PCA

In [54]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

crypto_pca = pca.fit_transform(crypto_scaled)

In [55]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"]
)
pca_df.head()

,PC 1,PC 2,PC 3
0,-0.254379,-0.094993,-0.001305
1,0.417085,-0.042052,-0.002038
2,2.106930,0.073820,-0.001584
3,1.080645,0.001401,-0.002079
4,2.532252,0.109538,-0.001278


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [56]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\matth\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [57]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=0)
# Fit the model
model.fit(pca_df)
# Predict clusters
predictions = model.predict(pca_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
pca_df['class'] = model.labels_

df_merged = crypto_df.join(coinname_df, how='inner')
df_merged = df_merged.append(pca_df)

df_merged#May need to reorder columns

,Algorithm,ProofType,TotalCoinsMined,MaxSupply,CoinName,PC 1,PC 2,PC 3,class
NSR,PoS,PoS,6168467233.8311,0,NuShares,NaN,NaN,NaN,NaN
TRI,X13,PoW/PoS,190083.632215,0,Triangles Coin,NaN,NaN,NaN,NaN
CMTC,Scrypt,PoW,872830,0,CometCoin,NaN,NaN,NaN,NaN
CHAT,Scrypt,PoW/PoS,1000000000,-1,OpenChat,NaN,NaN,NaN,NaN
QRL,RandomX,PoW,75321347.306076,105000000,Quantum Resistant Ledger,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
290,NaN,NaN,NaN,NaN,NaN,2.424125,0.101508,-0.001391,1.0
291,NaN,NaN,NaN,NaN,NaN,0.543856,0.015661,-0.000566,1.0
292,NaN,NaN,NaN,NaN,NaN,-1.106351,-0.151842,-0.000402,0.0
293,NaN,NaN,NaN,NaN,NaN,0.565691,-0.035676,-0.002280,1.0


### Visualizing Results

#### 3D-Scatter with Clusters

In [58]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    df_merged,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name='CoinName',
    hover_data= ['Algorithm'],
    color="class",
    symbol="class",
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

KeyError: (nan, '')

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
